In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer   # TfidfVectorizer
from sklearn.metrics.pairwise import paired_cosine_distances
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import hstack, vstack

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Ridge
from sklearn.preprocessing import normalize
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
# Исходная обучающая выборка
data = pd.read_csv('../input/train.csv')
# Исходна тестовая выборка
data_test_b = pd.read_csv('../input/test.csv')
stop = set(stopwords.words('english'))

In [ ]:
data.question1=data.question1.fillna('').astype(str)
data.question2=data.question2.fillna('').astype(str)

In [ ]:
def texts_vectorization(vectorizer, texts_1, texts_2):
    vectorizer.fit(pd.concat([texts_1.fillna(''), texts_2.fillna('')]))
    vect_1 = vectorizer.transform(texts_1.fillna(''))
    vect_2 = vectorizer.transform(texts_2.fillna(''))
    return vect_1, vect_2
def stopwords_go_away(Question):
    global stop
    if type(Question)==str:
        for mark in [',', '.', '!', ':','&', '?', '-', '[', ']', '{', '}', '(', ')', '/', '^', '"',"'"]:
            Question=Question.replace(mark,' ')
        words=[i for i in Question.lower().split() if i not in stop]
        return ' '.join(words)
data['question1stop']= data['question1'].apply(stopwords_go_away)
data['question2stop']= data['question2'].apply(stopwords_go_away)

In [ ]:
def doublewords_go_away(Question):
    if type(Question)==str:
        words=list(set(Question.lower().split()))
        return ' '.join(words)
data['question1wd']= data['question1'].apply(doublewords_go_away)
data['question2wd']= data['question2'].apply(doublewords_go_away)
data['question1min']= data['question1wd'].apply(doublewords_go_away)
data['question2min']= data['question2wd'].apply(doublewords_go_away)

In [ ]:
vect = CountVectorizer(lowercase=True, analyzer='word')
bag_product, bag_search = texts_vectorization(vect, data.question1, data.question2)
data['cosine_similarity_queries'] = paired_cosine_distances(bag_product, bag_search)

In [ ]:
from sklearn.linear_model import LinearRegression
def score_on_features(data, features):
    for clf in [LogisticRegression()]:
        X = data[features]
        res = cross_val_score(clf, X, y=data.is_duplicate, scoring='neg_log_loss', cv=10)
        print (res.mean(), res.std())

In [ ]:
features_1 = ['cosine_similarity_queries']
score_on_features(data, features_1)

In [ ]:
tsvd = TruncatedSVD(n_components=10)
vert_queiries = vstack([bag_product, bag_search])
vert_svd = tsvd.fit_transform(vert_queiries)
product_vert_svd, search_vert_svd = vert_svd[:bag_product.shape[0], :], vert_svd[bag_product.shape[0]:, :]
features_2 = []
for num_comp in range(product_vert_svd.shape[1]):
    f_name = 'comp_svd_diff_{}'.format(num_comp)
    features_2.append(f_name)
    data[f_name] = np.abs(product_vert_svd[:, num_comp] - search_vert_svd[:, num_comp])

In [ ]:
tsvd = TruncatedSVD(n_components=10)
side_queries = hstack([bag_product, bag_search])
svd_features = tsvd.fit_transform(side_queries)
features_3 = []
for num_comp in range(svd_features.shape[1]):
    f_name = 'svd_paired_feat_{}'.format(num_comp)
    features_3.append(f_name)
    data[f_name] = svd_features[:, num_comp]

In [ ]:
data['len1'] = data['question1'].fillna(0).str.len().astype(np.float32)
data['len2'] = data['question2'].fillna(0).str.len().astype(np.float32)

In [ ]:
data['abs_diff_len1_len2'] = np.abs(data['len1'] - data['len2'])
features_4=['abs_diff_len1_len2']

In [ ]:
data.abs_diff_len1_len2=data.abs_diff_len1_len2.fillna(100000)

In [ ]:
score_on_features(data, features_1+features_2+features_3+features_4)

In [ ]:
tsvd = TruncatedSVD(n_components=15)
bag_product.shape, bag_search.shape
vert_queiries = vstack([bag_product, bag_search])
vert_queiries.shape
vert_svd = tsvd.fit_transform(vert_queiries)
product_vert_svd, search_vert_svd = vert_svd[:bag_product.shape[0], :], vert_svd[bag_product.shape[0]:, :]
product_vert_svd.shape, product_vert_svd.shape

In [ ]:
features_2 = []
for num_comp in range(product_vert_svd.shape[1]):
    f_name = 'comp_svd_diff_{}'.format(num_comp)
    features_2.append(f_name)
    data[f_name] = np.abs(product_vert_svd[:, num_comp] - search_vert_svd[:, num_comp])

In [ ]:
data

In [ ]:
tsvd = TruncatedSVD(n_components=10)
side_queries = hstack([bag_product, bag_search])
svd_features = tsvd.fit_transform(side_queries)
features_3 = []
for num_comp in range(svd_features.shape[1]):
    f_name = 'svd_paired_feat_{}'.format(num_comp)
    features_3.append(f_name)
    data[f_name] = svd_features[:, num_comp]

In [ ]:
score_on_features(data, features_1+features_2+features_3)

In [ ]:
data['len1'] = data['question1'].str.len().astype(np.float32)
data['len2'] = data['question2'].str.len().astype(np.float32)
data['abs_diff_len1_len2'] = (np.abs(data['len1'] - data['len2']))

In [ ]:
data['len1'] = data['question1'].str.len().astype(np.float32)
data['len2'] = data['question2'].str.len().astype(np.float32)
data['abs_diff_len1_len2'] = (np.abs(data['len1'] - data['len2']))
data['len1min'] = data['question1min'].str.len().astype(np.float32)
data['len2min'] = data['question2min'].str.len().astype(np.float32)
data['abs_diff_len1_len2min'] = (np.abs(data['len1min'] - data['len2min']))
features_4=['abs_diff_len1_len2','abs_diff_len1_len2min' ]
score_on_features(data, features_1+features_2+features_3+features_4)

In [ ]:
from sklearn.metrics import jaccard_similarity_score
def jaccard(X, Y):
    res=np.zeros(X.shape[0])
    for i in range(X.shape[0]):
        X1=set(X[i].split(' '))
        Y1=set(Y[i].split(' '))
        if len((X1 | Y1) - (X1 &Y1))==0:
            res[i]=1.0
        else:
            res[i]=1.0*len(X1 &Y1)/len((X1 | Y1)) #- (X1 &Y1)
    return res
data['jaccard']=jaccard(data.question1.values, data.question2.values)

In [ ]:
features_5=['jaccard' ]
score_on_features(data, features_1+features_2+features_3+features_4+features_5)

In [ ]:
data

In [ ]:
X=data.drop('')

In [ ]:
model=LogisticRegression().fit(, data['is_duplicate'])

In [ ]:
data=data_test_b
data

In [ ]:
def texts_vectorization(vectorizer, texts_1, texts_2):
    vectorizer.fit(pd.concat([texts_1.fillna(''), texts_2.fillna('')]))
    vect_1 = vectorizer.transform(texts_1.fillna(''))
    vect_2 = vectorizer.transform(texts_2.fillna(''))
    return vect_1, vect_2
def stopwords_go_away(Question):
    global stop
    if type(Question)==str:
        for mark in [',', '.', '!', ':','&', '?', '-', '[', ']', '{', '}', '(', ')', '/', '^', '"',"'"]:
            Question=Question.replace(mark,' ')
        words=[i for i in Question.lower().split() if i not in stop]
        return ' '.join(words)
data.question1= data['question1'].apply(stopwords_go_away)
data.question2= data['question2'].apply(stopwords_go_away)
def doublewords_go_away(Question):
    if type(Question)==str:
        words=list(set(Question.lower().split()))
        return ' '.join(words)
data.question1= data['question1'].apply(doublewords_go_away)
data.question2= data['question2'].apply(doublewords_go_away)
vect = CountVectorizer(lowercase=True, analyzer='word')
bag_product, bag_search = texts_vectorization(vect, data.question1, data.question2)
data['is_duplicate'] = paired_cosine_distances(bag_product, bag_search)
data['len1'] = data['question1'].str.len().astype(np.float32)
data['len2'] = data['question2'].str.len().astype(np.float32)
data['abs_diff_len1_len2'] = (np.abs(data['len1'] - data['len2']))
data['len1min'] = data['question1min'].str.len().astype(np.float32)
data['len2min'] = data['question2min'].str.len().astype(np.float32)
data['abs_diff_len1_len2min'] = (np.abs(data['len1min'] - data['len2min']))
features_4=['abs_diff_len1_len2','abs_diff_len1_len2min' ]
data['jaccard']=jaccard(data.question1.values, data.question2.values)
features_5=['jaccard' ]
tsvd = TruncatedSVD(n_components=10)
vert_queiries = vstack([bag_product, bag_search])
vert_svd = tsvd.fit_transform(vert_queiries)
product_vert_svd, search_vert_svd = vert_svd[:bag_product.shape[0], :], vert_svd[bag_product.shape[0]:, :]
features_2 = []
for num_comp in range(product_vert_svd.shape[1]):
    f_name = 'comp_svd_diff_{}'.format(num_comp)
    features_2.append(f_name)
    data[f_name] = np.abs(product_vert_svd[:, num_comp] - search_vert_svd[:, num_comp])
tsvd = TruncatedSVD(n_components=10)
side_queries = hstack([bag_product, bag_search])
svd_features = tsvd.fit_transform(side_queries)
features_3 = []
for num_comp in range(svd_features.shape[1]):
    f_name = 'svd_paired_feat_{}'.format(num_comp)
    features_3.append(f_name)
    data[f_name] = svd_features[:, num_comp]


In [ ]:
data

In [ ]:
data

In [ ]:
data['is_duplicate']=0.2

In [ ]:
X=data.fillna(0).drop([''])
clf= LogisticRegression()
data.is_duplicate=model.predict()

In [ ]:
data.index=data.test_id
data.drop(['question1', 'question2', 'test_id'], axis=1).to_csv('submition1')